# Inerpretation-Net Training

## Specitication of Experiment Settings

In [1]:
import sys
print(sys.version)


3.9.6 (default, Aug 18 2021, 19:38:01) 
[GCC 7.5.0]


In [2]:
#######################################################################################################################################
###################################################### CONFIG FILE ####################################################################
#######################################################################################################################################
sleep_time = 0 #minutes



config = {
    'function_family': {
        'maximum_depth': 4,
        'beta': 1,
        'decision_sparsity': 1,
        'fully_grown': True,                      
    },
    'data': {
        'number_of_variables': 5, 
        'num_classes': 2,
        
        'function_generation_type': 'random_decision_tree', # 'make_classification' 'random_decision_tree'
        'objective': 'classification', # 'regression'
        
        'x_max': 1,
        'x_min': 0,
        'x_distrib': 'uniform', #'normal', 'uniform',       
                
        'lambda_dataset_size': 1000, #number of samples per function
        #'number_of_generated_datasets': 10000,
        
        'noise_injected_level': 0, 
        'noise_injected_type': 'flip_percentage', # '' 'normal' 'uniform' 'normal_range' 'uniform_range'
    }, 
    'lambda_net': {
        'epochs_lambda': 1000,
        'early_stopping_lambda': True, 
        'early_stopping_min_delta_lambda': 1e-2,
        'batch_lambda': 64,
        'dropout_lambda': 0,
        'lambda_network_layers': [64],
        'optimizer_lambda': 'adam',
        'loss_lambda': 'binary_crossentropy', #categorical_crossentropy
        
        'number_of_lambda_weights': None,
        
        'number_initializations_lambda': 1, 
        
        'number_of_trained_lambda_nets': 10000,
    },     
    
    'i_net': {
        'dense_layers': [2048],
        'convolution_layers': None,
        'lstm_layers': None,
        'dropout': [0.2],
        
        'optimizer': 'adam', #adam
        'learning_rate': 0.01,
        'loss': 'binary_crossentropy',
        'metrics': ['binary_accuracy'],
        
        'epochs': 10, 
        'early_stopping': True,
        'batch_size': 256,

        'interpretation_dataset_size': 1000,
                
        'test_size': 50, #Float for fraction, Int for number 0
        
        'function_representation_type': 3, # 1=standard representation; 2=sparse representation, 3=vanilla_dt

        'optimize_decision_function': True, #False
        'function_value_loss': True, #False
                      
        'data_reshape_version': None, #default to 2 options:(None, 0,1 2)
        
        'nas': False,
        'nas_type': 'SEQUENTIAL', #options:(None, 'SEQUENTIAL', 'CNN', 'LSTM', 'CNN-LSTM', 'CNN-LSTM-parallel')      
        'nas_trials': 100,
    },    
    
    'evaluation': {   
        #'inet_holdout_seed_evaluation': False,
            
        'random_evaluation_dataset_size': 500, 
        'per_network_optimization_dataset_size': 5000,

        'sklearn_dt_benchmark': False,
        'sdt_benchmark': False,
        
    },    
    
    'computation':{
        'load_model': False,
        
        'n_jobs': -3,
        'use_gpu': False,
        'gpu_numbers': '0',
        'RANDOM_SEED': 42,   
    }
}


## Imports

In [3]:
#######################################################################################################################################
########################################### IMPORT GLOBAL VARIABLES FROM CONFIG #######################################################
#######################################################################################################################################
globals().update(config['function_family'])
globals().update(config['data'])
globals().update(config['lambda_net'])
globals().update(config['i_net'])
globals().update(config['evaluation'])
globals().update(config['computation'])

In [4]:
#######################################################################################################################################
##################################################### IMPORT LIBRARIES ################################################################
#######################################################################################################################################
from itertools import product       
from tqdm import tqdm_notebook as tqdm
import pickle
import numpy as np
import pandas as pd
import scipy as sp
import timeit
import psutil

from functools import reduce
from more_itertools import random_product 
from sklearn.preprocessing import Normalizer

import sys
import os
import shutil

import logging

#from prettytable import PrettyTable
#import colored
import math

import time
from datetime import datetime
from collections.abc import Iterable


from joblib import Parallel, delayed

from scipy.integrate import quad

from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold, KFold, ParameterGrid, ParameterSampler
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, f1_score, mean_absolute_error, r2_score, log_loss
from sklearn.tree import DecisionTreeClassifier, plot_tree



#from similaritymeasures import frechet_dist, area_between_two_curves, dtw
import tensorflow as tf
#import tensorflow_addons as tfa
import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


import tensorflow.keras.backend as K
from livelossplot import PlotLossesKerasTF
#from keras_tqdm import TQDMNotebookCallback

from matplotlib import pyplot as plt
import seaborn as sns


import random 


import warnings

from IPython.display import Image
from IPython.display import display, Math, Latex, clear_output



2021-09-06 20:23:49.865247: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:
2021-09-06 20:23:49.865273: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
tf.__version__

'2.6.0'

In [6]:
#######################################################################################################################################
########################################### IMPORT GLOBAL VARIABLES FROM CONFIG #######################################################
#######################################################################################################################################
globals().update(config['function_family'])
globals().update(config['data'])
globals().update(config['lambda_net'])
globals().update(config['evaluation'])
globals().update(config['computation'])

In [7]:
#######################################################################################################################################
################################################### VARIABLE ADJUSTMENTS ##############################################################
#######################################################################################################################################

config['i_net']['data_reshape_version'] = 2 if data_reshape_version == None and (convolution_layers != None or lstm_layers != None or (nas and nas_type != 'SEQUENTIAL')) else data_reshape_version

#######################################################################################################################################
###################################################### SET VARIABLES + DESIGN #########################################################
#######################################################################################################################################

#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_numbers if use_gpu else ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
#os.environ['XLA_FLAGS'] =  '--xla_gpu_cuda_data_dir=/usr/lib/cuda-10.1'

logging.getLogger('tensorflow').disabled = True

sns.set_style("darkgrid")
#np.set_printoptions(suppress=True)

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
if int(tf.__version__[0]) >= 2:
    tf.random.set_seed(RANDOM_SEED)
else:
    tf.set_random_seed(RANDOM_SEED)
    
    
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 200)
np.set_printoptions(threshold=200)
np.set_printoptions(suppress=True)

warnings.filterwarnings('ignore')

In [8]:
from utilities.InterpretationNet import *
from utilities.LambdaNet import *
from utilities.metrics import *
from utilities.utility_functions import *
from utilities.DecisionTree_BASIC import *

#######################################################################################################################################
####################################################### CONFIG ADJUSTMENTS ############################################################
#######################################################################################################################################

config['lambda_net']['number_of_lambda_weights'] = get_number_of_lambda_net_parameters(lambda_network_layers, number_of_variables, num_classes)
config['function_family']['basic_function_representation_length'] = (2 ** maximum_depth - 1) * number_of_variables + (2 ** maximum_depth - 1) + (2 ** maximum_depth) * num_classes
config['function_family']['function_representation_length'] = ( (2 ** maximum_depth - 1) * number_of_variables + (2 ** maximum_depth - 1) + (2 ** maximum_depth) * num_classes  if function_representation_type == 1 
                                                              else (2 ** maximum_depth - 1) * decision_sparsity + (2 ** maximum_depth - 1) + ((2 ** maximum_depth - 1)  * decision_sparsity * number_of_variables) + (2 ** maximum_depth) * num_classes if function_representation_type == 2
                                                              else (2 ** maximum_depth - 1) * decision_sparsity + ((2 ** maximum_depth - 1)  * decision_sparsity * number_of_variables) + (2 ** maximum_depth))

#######################################################################################################################################
################################################## UPDATE VARIABLES ###################################################################
#######################################################################################################################################
globals().update(config['function_family'])
globals().update(config['data'])
globals().update(config['lambda_net'])
globals().update(config['i_net'])
globals().update(config['evaluation'])
globals().update(config['computation'])

#initialize_LambdaNet_config_from_curent_notebook(config)
#initialize_metrics_config_from_curent_notebook(config)
#initialize_utility_functions_config_from_curent_notebook(config)
#initialize_InterpretationNet_config_from_curent_notebook(config)


#######################################################################################################################################
###################################################### PATH + FOLDER CREATION #########################################################
#######################################################################################################################################
globals().update(generate_paths(config, path_type='interpretation_net'))
create_folders_inet(config)

#######################################################################################################################################
############################################################ SLEEP TIMER ##############################################################
#######################################################################################################################################
sleep_minutes(sleep_time)

In [9]:
print(path_identifier_interpretation_net)

print(path_identifier_lambda_net_data)


lNetSize1000_numLNets10000_var5_class2_random_decision_tree_xMax1_xMin0_xDistuniform_depth4_beta1_decisionSpars1_fullyGrown/64_e1000ES0.01_b64_drop0_adam_binary_crossentropy_fixedInit1-seed42/inet_dense2048_drop0.2e10b256_adam
lNetSize1000_numLNets10000_var5_class2_random_decision_tree_xMax1_xMin0_xDistuniform_depth4_beta1_decisionSpars1_fullyGrown/64_e1000ES0.01_b64_drop0_adam_binary_crossentropy_fixedInit1-seed42


In [10]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num XLA-GPUs Available: ", len(tf.config.experimental.list_physical_devices('XLA_GPU')))

Num GPUs Available:  0
Num XLA-GPUs Available:  0


2021-09-06 20:23:55.561076: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:
2021-09-06 20:23:55.561112: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


## Load Data and Generate Datasets

In [11]:
def load_lambda_nets(config, no_noise=False, n_jobs=1):
    
    #def generate_lambda_net()
    
    #if psutil.virtual_memory().percent > 80:
        #raise SystemExit("Out of RAM!")
    
    if no_noise==True:
        config['noise_injected_level'] = 0
    path_dict = generate_paths(config, path_type='interpretation_net')        
        
    directory = './data/weights/' + 'weights_' + path_dict['path_identifier_lambda_net_data'] + '/'
    path_network_parameters = directory + 'weights' + '.txt'
    path_X_data = directory + 'X_test_lambda.txt'
    path_y_data = directory + 'y_test_lambda.txt'        
    
    network_parameters = pd.read_csv(path_network_parameters, sep=",", header=None)
    network_parameters = network_parameters.sort_values(by=0)
    if no_noise == False:
        network_parameters = network_parameters.sample(n=config['i_net']['interpretation_dataset_size'], random_state=config['computation']['RANDOM_SEED'])
    
    X_test_lambda = pd.read_csv(path_X_data, sep=",", header=None)
    X_test_lambda = X_test_lambda.sort_values(by=0)
    if no_noise == False:
        X_test_lambda = X_test_lambda.sample(n=config['i_net']['interpretation_dataset_size'], random_state=config['computation']['RANDOM_SEED'])
    
    y_test_lambda = pd.read_csv(path_y_data, sep=",", header=None)
    y_test_lambda = y_test_lambda.sort_values(by=0)
    if no_noise == False:
        y_test_lambda = y_test_lambda.sample(n=config['i_net']['interpretation_dataset_size'], random_state=config['computation']['RANDOM_SEED'])
        
        
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky') #loky

    lambda_nets = parallel(delayed(LambdaNet)(network_parameters_row, 
                                              X_test_lambda_row, 
                                              y_test_lambda_row, 
                                              config) for network_parameters_row, X_test_lambda_row, y_test_lambda_row in zip(network_parameters.values, X_test_lambda.values, y_test_lambda.values))          
    del parallel
    
    base_model = generate_base_model(config)  
    
    def initialize_network_wrapper(config, lambda_net, base_model):
        lambda_net.initialize_network(config, base_model)
    
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='sequential')
    _ = parallel(delayed(initialize_network_wrapper)(config, lambda_net, base_model) for lambda_net in lambda_nets)   
    del parallel
    
    def initialize_target_function_wrapper(config, lambda_net):
        lambda_net.initialize_target_function(config)
    
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='sequential')
    _ = parallel(delayed(initialize_target_function_wrapper)(config, lambda_net) for lambda_net in lambda_nets)   
    del parallel
        
    
    #lambda_nets = [None] * network_parameters.shape[0]
    #for i, (network_parameters_row, X_test_lambda_row, y_test_lambda_row) in tqdm(enumerate(zip(network_parameters.values, X_test_lambda.values, y_test_lambda.values)), total=network_parameters.values.shape[0]):        
    #    lambda_net = LambdaNet(network_parameters_row, X_test_lambda_row, y_test_lambda_row, config)
    #    lambda_nets[i] = lambda_net
                
    lambda_net_dataset = LambdaNetDataset(lambda_nets)
        
    return lambda_net_dataset
    

In [12]:
#LOAD DATA
if noise_injected_level > 0:
    lambda_net_dataset_training = load_lambda_nets(config, no_noise=True, n_jobs=n_jobs)
    lambda_net_dataset_evaluation = load_lambda_nets(config, n_jobs=n_jobs)

    lambda_net_dataset_train, lambda_net_dataset_valid = split_LambdaNetDataset(lambda_net_dataset_training, test_split=0.1)
    _, lambda_net_dataset_test = split_LambdaNetDataset(lambda_net_dataset_evaluation, test_split=test_size)
    
else:
    lambda_net_dataset = load_lambda_nets(config, n_jobs=n_jobs)

    lambda_net_dataset_train_with_valid, lambda_net_dataset_test = split_LambdaNetDataset(lambda_net_dataset, test_split=test_size)
    lambda_net_dataset_train, lambda_net_dataset_valid = split_LambdaNetDataset(lambda_net_dataset_train_with_valid, test_split=0.1)

    

2021-09-06 20:23:55.561138: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dws-05): /proc/driver/nvidia/version does not exist
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 14 concurrent workers.
2021-09-06 20:24:00.412355: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:
2021-09-06 20:24:00.412379: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-06 20:24:00.421976: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:
2021-09-06 20:24

## Data Inspection

In [13]:
lambda_net_dataset_train.shape

(855, 573)

In [14]:
lambda_net_dataset_valid.shape

(95, 573)

In [15]:
lambda_net_dataset_test.shape

(50, 573)

In [16]:
lambda_net_dataset_train.as_pandas(config).head()

,index,seed,f0v0,f0v1,f0v2,f0v3,f0v4,f1v0,f1v1,f1v2,f1v3,f1v4,f2v0,f2v1,f2v2,f2v3,f2v4,f3v0,f3v1,f3v2,f3v3,f3v4,f4v0,f4v1,f4v2,f4v3,f4v4,f5v0,f5v1,f5v2,f5v3,f5v4,f6v0,f6v1,f6v2,f6v3,f6v4,f7v0,f7v1,f7v2,f7v3,f7v4,f8v0,f8v1,f8v2,f8v3,f8v4,f9v0,f9v1,f9v2,f9v3,f9v4,f10v0,f10v1,f10v2,f10v3,f10v4,f11v0,f11v1,f11v2,f11v3,f11v4,f12v0,f12v1,f12v2,f12v3,f12v4,f13v0,f13v1,f13v2,f13v3,f13v4,f14v0,f14v1,f14v2,f14v3,f14v4,b0,b1,b2,b3,b4,b5,b6,b7,b8,b9,b10,b11,b12,b13,b14,lp0c0,lp0c1,lp1c0,lp1c1,lp2c0,lp2c1,lp3c0,lp3c1,...,wb_349,wb_350,wb_351,wb_352,wb_353,wb_354,wb_355,wb_356,wb_357,wb_358,wb_359,wb_360,wb_361,wb_362,wb_363,wb_364,wb_365,wb_366,wb_367,wb_368,wb_369,wb_370,wb_371,wb_372,wb_373,wb_374,wb_375,wb_376,wb_377,wb_378,wb_379,wb_380,wb_381,wb_382,wb_383,wb_384,wb_385,wb_386,wb_387,wb_388,wb_389,wb_390,wb_391,wb_392,wb_393,wb_394,wb_395,wb_396,wb_397,wb_398,wb_399,wb_400,wb_401,wb_402,wb_403,wb_404,wb_405,wb_406,wb_407,wb_408,wb_409,wb_410,wb_411,wb_412,wb_413,wb_414,wb_415,wb_416,wb_417,wb_418,wb_419,wb_420,wb_421,wb_422,wb_423,wb_424,wb_425,wb_426,wb_427,wb_428,wb_429,wb_430,wb_431,wb_432,wb_433,wb_434,wb_435,wb_436,wb_437,wb_438,wb_439,wb_440,wb_441,wb_442,wb_443,wb_444,wb_445,wb_446,wb_447,wb_448
9644,9644.000,42,0.000,0.276,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.335,0.000,0.000,0.000,0.398,0.000,0.000,0.000,0.000,0.000,0.357,-0.421,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.289,-0.276,0.000,0.000,0.000,0.000,0.000,0.245,0.000,0.000,0.000,0.000,-0.196,0.000,0.000,0.000,0.397,0.000,0.000,0.000,0.000,0.000,0.000,-0.344,0.000,0.000,0.000,0.000,0.000,0.422,0.000,0.000,-0.410,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.390,0.000,-0.405,0.000,0.000,0.000,-0.143,-0.422,0.253,0.156,-0.355,0.256,-0.259,-0.106,0.306,0.373,0.238,0.201,-0.447,-0.164,-0.175,-0.008,0.092,0.213,0.156,0.213,0.178,0.187,0.175,...,0.175,-0.044,0.018,-0.046,-0.010,-0.018,-0.029,-0.065,-0.011,-0.036,0.000,-0.048,0.045,0.014,0.008,0.130,0.115,-0.000,0.235,0.054,0.124,-0.020,0.000,0.000,-0.010,-0.040,0.058,-0.045,-0.070,0.133,0.067,0.235,0.097,-0.072,-0.024,-0.473,0.507,0.500,-0.831,-0.768,0.071,0.770,-0.119,-0.688,-0.272,0.532,-0.601,-0.457,-0.147,0.414,0.364,-0.605,-0.583,-0.259,-0.143,-0.741,-0.534,-0.478,0.342,0.311,0.516,-0.063,0.725,-0.688,-0.476,-0.087,0.654,0.254,0.497,-0.153,0.211,0.349,0.528,0.206,-0.294,0.324,0.594,0.623,0.643,0.850,0.823,0.614,-0.848,0.570,-0.647,0.012,-0.187,-0.257,-0.152,-0.014,0.575,0.313,-0.121,-0.622,0.663,-0.626,0.734,-0.156,0.471,-0.021
36,36.000,42,0.000,0.000,0.000,-0.372,0.000,0.000,0.000,0.000,-0.365,0.000,-0.291,0.000,0.000,0.000,0.000,0.000,0.301,0.000,0.000,0.000,0.000,0.440,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.420,-0.331,0.000,0.000,0.000,0.000,0.000,0.393,0.000,0.000,0.000,0.000,0.000,0.137,0.000,0.000,0.000,0.000,0.000,0.000,-0.392,0.000,0.000,0.364,0.000,0.000,0.000,-0.387,0.000,0.000,0.000,0.354,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.431,0.000,0.000,0.000,0.000,-0.349,0.000,0.244,0.382,0.242,-0.222,-0.004,0.024,-0.351,-0.201,0.003,0.183,-0.028,0.292,0.274,0.043,0.371,-0.249,-0.036,0.177,0.215,-0.142,0.176,0.088,-0.157,...,-0.043,-0.197,-0.055,-0.066,0.006,-0.218,-0.011,0.615,0.140,-0.072,0.000,-0.119,0.489,-0.076,-0.068,0.548,-0.015,-0.007,-0.174,-0.046,-0.179,-0.042,0.000,0.000,0.000,0.561,0.368,-0.087,-0.172,-0.154,0.483,-0.172,0.500,-0.100,-0.115,-0.469,0.634,0.504,-0.888,-0.826,0.837,0.141,-0.205,-0.695,-0.272,0.286,-0.600,-0.442,-0.145,0.098,0.153,-0.698,-0.449,-0.292,-0.366,-0.755,-0.537,-0.475,-0.277,0.057,0.653,-0.344,0.871,-0.756,-0.013,-0.432,0.175,0.224,0.157,-0.718,0.092,0.826,0.617,0.012,-0.294,-0.055,0.810,0.156,0.147,0.977,0.174,0.215,-0.901,0.125,-0.592,0.126,-0.187,-0.257,-0.159,0.673,0.626,-0.049,-0.509,-0.450,0.850,-0.736,0.876,-0.138,0.190,0.116
1568,1568.000,42,0.000,0.000,-0.373,0.000,0.000,0.000,0.390,0.000,0.000,0.000,-0.361,0.000,0.000,0.000,0.000,0.000,0.359,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.376,0.000,0.000,0.000,-0.409,0.000,0.000,0.000,0.000,0.000,0.376,0

In [17]:
lambda_net_dataset_valid.as_pandas(config).head()

,index,seed,f0v0,f0v1,f0v2,f0v3,f0v4,f1v0,f1v1,f1v2,f1v3,f1v4,f2v0,f2v1,f2v2,f2v3,f2v4,f3v0,f3v1,f3v2,f3v3,f3v4,f4v0,f4v1,f4v2,f4v3,f4v4,f5v0,f5v1,f5v2,f5v3,f5v4,f6v0,f6v1,f6v2,f6v3,f6v4,f7v0,f7v1,f7v2,f7v3,f7v4,f8v0,f8v1,f8v2,f8v3,f8v4,f9v0,f9v1,f9v2,f9v3,f9v4,f10v0,f10v1,f10v2,f10v3,f10v4,f11v0,f11v1,f11v2,f11v3,f11v4,f12v0,f12v1,f12v2,f12v3,f12v4,f13v0,f13v1,f13v2,f13v3,f13v4,f14v0,f14v1,f14v2,f14v3,f14v4,b0,b1,b2,b3,b4,b5,b6,b7,b8,b9,b10,b11,b12,b13,b14,lp0c0,lp0c1,lp1c0,lp1c1,lp2c0,lp2c1,lp3c0,lp3c1,...,wb_349,wb_350,wb_351,wb_352,wb_353,wb_354,wb_355,wb_356,wb_357,wb_358,wb_359,wb_360,wb_361,wb_362,wb_363,wb_364,wb_365,wb_366,wb_367,wb_368,wb_369,wb_370,wb_371,wb_372,wb_373,wb_374,wb_375,wb_376,wb_377,wb_378,wb_379,wb_380,wb_381,wb_382,wb_383,wb_384,wb_385,wb_386,wb_387,wb_388,wb_389,wb_390,wb_391,wb_392,wb_393,wb_394,wb_395,wb_396,wb_397,wb_398,wb_399,wb_400,wb_401,wb_402,wb_403,wb_404,wb_405,wb_406,wb_407,wb_408,wb_409,wb_410,wb_411,wb_412,wb_413,wb_414,wb_415,wb_416,wb_417,wb_418,wb_419,wb_420,wb_421,wb_422,wb_423,wb_424,wb_425,wb_426,wb_427,wb_428,wb_429,wb_430,wb_431,wb_432,wb_433,wb_434,wb_435,wb_436,wb_437,wb_438,wb_439,wb_440,wb_441,wb_442,wb_443,wb_444,wb_445,wb_446,wb_447,wb_448
3501,3501.000,42,0.000,0.384,0.000,0.000,0.000,0.000,0.000,-0.437,0.000,0.000,0.000,0.000,0.000,0.000,-0.433,0.000,-0.373,0.000,0.000,0.000,0.000,0.000,0.369,0.000,0.000,0.000,0.446,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.392,0.000,0.000,0.000,0.410,0.000,0.000,-0.289,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.302,0.000,-0.440,0.000,0.000,0.000,-0.372,0.000,0.000,0.000,0.000,0.000,0.387,0.000,0.000,0.000,-0.411,0.000,0.000,0.000,0.000,0.000,0.000,-0.346,0.000,0.000,-0.445,0.008,-0.091,0.436,0.101,-0.341,-0.073,-0.221,0.239,-0.342,0.444,-0.048,0.300,0.314,-0.107,-0.004,-0.078,-0.049,-0.010,-0.223,-0.105,-0.092,0.153,...,0.313,-0.043,0.253,0.305,0.140,-0.028,0.142,0.400,0.008,0.255,0.000,0.112,0.234,0.165,0.164,0.201,0.358,0.211,-0.055,0.225,0.044,0.020,0.000,0.000,-0.011,-0.043,-0.094,0.082,-0.067,-0.096,0.286,0.056,0.283,-0.073,0.183,-0.079,-0.521,0.212,-0.258,-0.210,1.027,1.337,-0.714,-0.762,-0.272,0.184,-0.719,-0.578,-0.154,0.368,1.341,0.847,-0.136,0.531,-0.145,-0.852,-0.638,-0.565,0.572,0.531,1.164,-0.611,0.838,-0.776,0.652,-0.089,0.753,1.648,0.637,-0.089,0.657,1.630,0.604,0.590,-0.294,0.370,0.809,0.697,0.700,1.249,1.602,0.700,-0.262,0.667,-0.838,-0.064,-0.187,-0.257,-0.150,-0.017,0.081,0.486,-0.122,-0.539,0.835,-0.728,1.051,-0.160,0.621,0.155
5748,5748.000,42,0.000,-0.440,0.000,0.000,0.000,0.000,0.000,0.000,0.422,0.000,0.339,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.337,0.000,0.000,0.000,0.000,0.000,0.438,0.000,0.000,0.000,0.000,-0.414,0.000,0.000,-0.390,0.000,0.000,0.000,0.403,0.000,0.000,0.000,0.000,0.000,0.000,0.378,0.000,0.000,0.000,0.441,0.000,0.000,-0.401,0.000,0.000,0.000,0.000,-0.436,0.000,0.000,0.000,0.000,0.000,0.000,-0.306,0.000,0.000,0.402,0.000,0.000,0.000,0.000,-0.428,0.000,0.000,0.000,0.000,-0.441,-0.077,0.004,-0.134,-0.220,0.153,0.392,-0.402,0.221,-0.126,0.176,-0.119,0.170,0.029,-0.209,0.176,-0.051,0.225,-0.163,0.113,0.232,0.006,-0.027,...,0.226,0.033,0.082,0.185,-0.048,0.074,0.012,0.248,0.124,-0.064,0.000,-0.014,0.152,0.123,0.134,0.229,0.205,0.008,0.102,-0.090,0.067,0.122,0.000,0.000,0.000,0.063,0.118,-0.037,0.077,0.032,0.139,-0.033,0.203,0.081,-0.010,-0.431,-0.336,0.409,-0.668,-0.641,0.464,0.604,-0.289,-0.238,-0.272,0.557,-0.455,0.387,-0.575,0.384,0.650,0.462,-0.504,-0.262,-0.507,-0.430,-0.042,-0.345,-0.215,0.110,0.424,-0.438,0.124,-0.305,0.452,-0.446,0.499,0.666,0.037,-0.500,0.121,0.537,0.543,0.019,-0.294,0.026,0.523,0.542,0.537,0.705,0.648,0.295,-0.685,0.089,-0.573,0.491,-0.187,-0.257,-0.159,-0.414,0.514,-0.046,-0.505,-0.473,0.545,-0.105,0.625,-0.551,0.327,-0.012
8103,8103.000,42,0.000,0.000,0.360,0.000,0.000,-0.264,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.442,0.000,0.000,0.000,0.000,0.000,0.392,0.000,0.000,0.000,0.000,0.316,0.000,0.000,0.000,-0.409,0.000,-0.406,0.000,0.000,0.000,0.000,0.000,0.000,0.000

In [18]:
lambda_net_dataset_test.as_pandas(config).head()

,index,seed,f0v0,f0v1,f0v2,f0v3,f0v4,f1v0,f1v1,f1v2,f1v3,f1v4,f2v0,f2v1,f2v2,f2v3,f2v4,f3v0,f3v1,f3v2,f3v3,f3v4,f4v0,f4v1,f4v2,f4v3,f4v4,f5v0,f5v1,f5v2,f5v3,f5v4,f6v0,f6v1,f6v2,f6v3,f6v4,f7v0,f7v1,f7v2,f7v3,f7v4,f8v0,f8v1,f8v2,f8v3,f8v4,f9v0,f9v1,f9v2,f9v3,f9v4,f10v0,f10v1,f10v2,f10v3,f10v4,f11v0,f11v1,f11v2,f11v3,f11v4,f12v0,f12v1,f12v2,f12v3,f12v4,f13v0,f13v1,f13v2,f13v3,f13v4,f14v0,f14v1,f14v2,f14v3,f14v4,b0,b1,b2,b3,b4,b5,b6,b7,b8,b9,b10,b11,b12,b13,b14,lp0c0,lp0c1,lp1c0,lp1c1,lp2c0,lp2c1,lp3c0,lp3c1,...,wb_349,wb_350,wb_351,wb_352,wb_353,wb_354,wb_355,wb_356,wb_357,wb_358,wb_359,wb_360,wb_361,wb_362,wb_363,wb_364,wb_365,wb_366,wb_367,wb_368,wb_369,wb_370,wb_371,wb_372,wb_373,wb_374,wb_375,wb_376,wb_377,wb_378,wb_379,wb_380,wb_381,wb_382,wb_383,wb_384,wb_385,wb_386,wb_387,wb_388,wb_389,wb_390,wb_391,wb_392,wb_393,wb_394,wb_395,wb_396,wb_397,wb_398,wb_399,wb_400,wb_401,wb_402,wb_403,wb_404,wb_405,wb_406,wb_407,wb_408,wb_409,wb_410,wb_411,wb_412,wb_413,wb_414,wb_415,wb_416,wb_417,wb_418,wb_419,wb_420,wb_421,wb_422,wb_423,wb_424,wb_425,wb_426,wb_427,wb_428,wb_429,wb_430,wb_431,wb_432,wb_433,wb_434,wb_435,wb_436,wb_437,wb_438,wb_439,wb_440,wb_441,wb_442,wb_443,wb_444,wb_445,wb_446,wb_447,wb_448
5206,5206.000,42,0.000,0.000,-0.419,0.000,0.000,0.000,0.000,0.000,0.000,0.396,0.000,0.353,0.000,0.000,0.000,-0.350,0.000,0.000,0.000,0.000,-0.423,0.000,0.000,0.000,0.000,0.000,0.000,0.422,0.000,0.000,0.000,0.000,-0.437,0.000,0.000,0.000,0.000,0.000,-0.297,0.000,0.429,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.297,0.000,0.442,0.000,0.000,0.000,0.000,0.000,-0.438,0.000,0.000,0.000,-0.399,0.000,0.000,0.000,0.000,0.000,0.404,0.000,0.000,0.000,0.000,-0.360,0.000,0.000,-0.115,0.129,0.102,0.301,-0.279,-0.022,0.078,0.193,-0.040,0.058,-0.309,0.393,-0.411,0.212,-0.024,0.224,0.034,0.072,0.034,0.044,-0.105,0.225,0.130,...,0.185,0.138,-0.078,-0.069,-0.045,0.205,0.041,0.144,0.078,0.015,0.000,-0.046,-0.047,-0.099,-0.088,0.147,-0.025,-0.045,0.251,-0.097,0.193,0.036,0.000,0.000,0.255,0.198,0.031,-0.038,0.185,-0.129,-0.001,0.210,-0.073,0.209,-0.046,-0.712,0.601,0.704,-1.003,-0.972,0.736,0.133,-0.809,-0.898,-0.272,0.837,-0.656,-0.671,-0.959,0.242,0.134,-0.909,-0.138,0.530,-0.847,-0.962,-0.372,0.649,0.541,0.648,0.689,-0.730,0.147,-1.008,-0.641,-0.804,0.165,0.210,0.131,-0.774,0.676,0.824,0.843,0.601,-0.294,0.534,0.081,0.154,0.143,0.997,0.164,0.535,-1.040,0.099,-0.962,0.740,-0.187,-0.257,-0.942,-0.693,0.768,0.580,-0.762,-0.104,0.853,-0.938,0.178,-0.922,0.593,-0.045
2771,2771.000,42,0.000,0.376,0.000,0.000,0.000,0.000,0.000,0.000,0.392,0.000,0.000,0.416,0.000,0.000,0.000,0.000,0.000,0.000,0.362,0.000,0.000,0.000,-0.321,0.000,0.000,0.000,0.393,0.000,0.000,0.000,0.000,0.000,0.000,0.209,0.000,0.000,0.000,0.000,0.000,0.409,0.000,0.000,0.000,0.376,0.000,0.270,0.000,0.000,0.000,0.000,0.000,-0.330,0.000,0.000,0.000,0.000,-0.381,0.000,0.000,0.000,0.000,0.000,0.000,0.330,0.000,0.000,-0.329,0.000,0.000,0.000,0.000,0.000,0.000,-0.372,0.000,-0.256,0.068,-0.087,-0.175,0.306,-0.043,-0.170,0.205,0.146,-0.445,0.374,0.416,-0.147,-0.137,0.145,0.038,0.157,-0.025,0.140,-0.100,0.012,0.179,-0.074,...,-0.442,0.435,-0.443,0.019,-0.467,-0.030,-0.482,0.441,-0.013,-0.435,0.000,-0.585,-0.182,-0.443,-0.454,0.355,0.398,-0.412,-0.061,-0.427,0.242,-0.451,0.000,0.000,0.540,0.497,-0.098,-0.550,0.102,0.298,0.052,0.521,-0.077,0.444,-0.438,-0.065,-0.430,0.327,-0.787,-0.192,0.441,1.038,-0.655,-0.584,-0.272,0.483,-0.428,0.773,-0.754,0.809,0.759,0.926,-0.602,0.578,-0.742,-0.632,-0.458,-0.495,0.680,0.695,0.432,-0.637,0.116,-0.682,0.797,-0.634,1.008,0.587,0.784,-0.083,0.785,1.114,0.381,0.705,-0.294,0.664,0.511,0.966,0.991,1.082,1.072,0.848,-0.250,0.823,-0.726,0.787,-0.187,-0.257,-0.814,-0.577,0.122,0.659,-0.412,-0.311,0.320,-0.766,0.177,-0.646,0.864,-0.270
5928,5928.000,42,0.387,0.000,0.000,0.000,0.000,-0.416,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.227,0.000,0.000,0.426,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.358,0.340,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.352,-0.440,0.000

## Interpretation Network Training

In [19]:
#%load_ext autoreload

In [20]:
#%autoreload 2
((X_valid, y_valid), 
 (X_test, y_test),
 history,

 model) = interpretation_net_training(
                                      lambda_net_dataset_train, 
                                      lambda_net_dataset_valid, 
                                      lambda_net_dataset_test,
                                      config,
                                      #callback_names=['plot_losses']
                                     )



----------------------------------------------- TRAINING INTERPRETATION NET -----------------------------------------------


2021-09-06 20:24:25.692800: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
4/4 [==============================] - 12s 1s/step - loss: 0.6711 - binary_accuracy_inet_decision_function_fv_metric: 0.5803 - val_loss: 0.6306 - val_binary_accuracy_inet_decision_function_fv_metric: 0.6481
Epoch 2/10
4/4 [==============================] - 0s 116ms/step - loss: 0.6259 - binary_accuracy_inet_decision_function_fv_metric: 0.6494 - val_loss: 0.6042 - val_binary_accuracy_inet_decision_function_fv_metric: 0.6731
Epoch 3/10
4/4 [==============================] - 0s 116ms/step - loss: 0.6155 - binary_accuracy_inet_decision_function_fv_metric: 0.6567 - val_loss: 0.5910 - val_binary_accuracy_inet_decision_function_fv_metric: 0.6843
Epoch 4/10
4/4 [==============================] - 0s 118ms/step - loss: 0.6003 - binary_accuracy_inet_decision_function_fv_metric: 0.6752 - val_loss: 0.5778 - val_binary_accuracy_inet_decision_function_fv_metric: 0.6941
Epoch 5/10
4/4 [==============================] - 0s 117ms/step - loss: 0.5906 - binary_accuracy_inet_decision_function_fv

2021-09-06 20:24:43.963823: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Training Time: 0:00:20
---------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------ LOADING MODELS -----------------------------------------------------
Loading Time: 0:00:01


<Figure size 432x288 with 0 Axes>

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 449)]        0                                            
__________________________________________________________________________________________________
hidden1_2048 (Dense)            (None, 2048)         921600      input[0][0]                      
__________________________________________________________________________________________________
activation1_relu (Activation)   (None, 2048)         0           hidden1_2048[0][0]               
__________________________________________________________________________________________________
dropout1_0.2 (Dropout)          (None, 2048)         0           activation1_relu[0][0]           
______________________________________________________________________________________________

In [23]:
y_test_inet_vanilla_dt_list = []
y_test_distilled_sklearn_vanilla_dt_list = []

binary_crossentropy_distilled_sklearn_vanilla_dt_list =[]
accuracy_distilled_sklearn_vanilla_dt_list = []
f1_score_distilled_sklearn_vanilla_dt_list = []

binary_crossentropy_inet_vanilla_dt_list =[]
accuracy_inet_vanilla_dt_list = []
f1_score_inet_vanilla_dt_list = []

number = 10

for lambda_net_parameters, lambda_net, X_test_lambda, y_test_lambda in tqdm(zip(lambda_net_dataset_train.network_parameters_array[:number], lambda_net_dataset_train.network_list[:number], lambda_net_dataset_train.X_test_lambda_array[:number], lambda_net_dataset_train.y_test_lambda_array[:number]), total=lambda_net_dataset_train.y_test_lambda_array[:number].shape[0]):
    dt_inet = model.predict(np.array([lambda_net_parameters]))[0]
    
    X_data_random = generate_random_data_points_custom(0, 1, 5000, 5)
    y_data_random_lambda_pred = lambda_net.predict(X_data_random)
    y_data_random_lambda_pred = np.round(y_data_random_lambda_pred).astype(np.int64)
    
    dt_sklearn_distilled = DecisionTreeClassifier(max_depth=config['function_family']['maximum_depth'])
    dt_sklearn_distilled.fit(X_data_random, y_data_random_lambda_pred)
    
    y_test_inet_vanilla_dt  = calculate_function_value_from_vanilla_decision_tree_parameters_wrapper(X_test_lambda, config)(dt_inet).numpy()
    y_test_distilled_sklearn_vanilla_dt = dt_sklearn_distilled.predict(X_test_lambda)
    
    y_test_inet_vanilla_dt_list.append(y_test_inet_vanilla_dt)
    y_test_distilled_sklearn_vanilla_dt_list.append(y_test_distilled_sklearn_vanilla_dt)
    
    y_test_lambda_pred = lambda_net.predict(X_test_lambda)
    y_test_lambda_pred = np.round(y_test_lambda_pred)
    
    binary_crossentropy_distilled_sklearn_vanilla_dt = log_loss(y_test_lambda_pred, y_test_distilled_sklearn_vanilla_dt)
    accuracy_distilled_sklearn_vanilla_dt = accuracy_score(y_test_lambda_pred, np.round(y_test_distilled_sklearn_vanilla_dt))
    f1_score_distilled_sklearn_vanilla_dt = f1_score(y_test_lambda_pred, np.round(y_test_distilled_sklearn_vanilla_dt))
    
    binary_crossentropy_inet_vanilla_dt = log_loss(y_test_lambda_pred, y_test_inet_vanilla_dt)
    accuracy_inet_vanilla_dt = accuracy_score(y_test_lambda_pred, np.round(y_test_inet_vanilla_dt))
    f1_score_inet_vanilla_dt = f1_score(y_test_lambda_pred, np.round(y_test_inet_vanilla_dt))

    binary_crossentropy_distilled_sklearn_vanilla_dt_list.append(np.nan_to_num(binary_crossentropy_distilled_sklearn_vanilla_dt))
    accuracy_distilled_sklearn_vanilla_dt_list.append(np.nan_to_num(accuracy_distilled_sklearn_vanilla_dt))
    f1_score_distilled_sklearn_vanilla_dt_list.append(np.nan_to_num(f1_score_distilled_sklearn_vanilla_dt))

    binary_crossentropy_inet_vanilla_dt_list.append(np.nan_to_num(binary_crossentropy_inet_vanilla_dt))
    accuracy_inet_vanilla_dt_list.append(np.nan_to_num(accuracy_inet_vanilla_dt))
    f1_score_inet_vanilla_dt_list.append(np.nan_to_num(f1_score_inet_vanilla_dt))
    
    
print('Binary Crossentropy:\t\t', np.round(np.mean(binary_crossentropy_distilled_sklearn_vanilla_dt_list), 3), '(Sklearn DT)' , '\t', np.round(np.mean(binary_crossentropy_inet_vanilla_dt_list), 3), '(I-Net DT)')
print('Accuracy:\t\t', np.round(np.mean(accuracy_distilled_sklearn_vanilla_dt_list), 3), '(Sklearn DT)' , '\t', np.round(np.mean(accuracy_inet_vanilla_dt_list), 3), '(I-Net DT)')
print('F1 Score:\t\t', np.round(np.mean(f1_score_distilled_sklearn_vanilla_dt_list), 3), '(Sklearn DT)' , '\t', np.round(np.mean(f1_score_inet_vanilla_dt_list), 3), '(I-Net DT)')
      

  0%|          | 0/10 [00:00<?, ?it/s]

Binary Crossentropy:		 2.653 (Sklearn DT) 	 0.571 (I-Net DT)
Accuracy:		 0.923 (Sklearn DT) 	 0.691 (I-Net DT)
F1 Score:		 0.91 (Sklearn DT) 	 0.572 (I-Net DT)
